In [39]:
import pandas as pd
import geopandas as gpd
import json

In [40]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [33]:
centroids = world.centroid
centroid_list = pd.concat([world.name, centroids], axis=1)
centroid_list.head(3)

<ipython-input-33-a002268788d0>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = world.centroid


name                            0
0       Fiji  POINT (163.85316 -17.31631)
1   Tanzania    POINT (34.75299 -6.25773)
2  W. Sahara   POINT (-12.13783 24.29117)

In [41]:
centroid_list = centroid_list.rename(columns={0: 'centroids'})

In [45]:
centroid_list[centroid_list['name'] == 'Syrian']

Empty DataFrame
Columns: [name, centroids]
Index: []

In [25]:
df = pd.read_csv('./global_power_plant_database - global_power_plant_database.csv')
df.head(3)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


country country_long                                          name  \
0     AFG  Afghanistan  Kajaki Hydroelectric Power Plant Afghanistan   
1     AFG  Afghanistan                                  Kandahar DOG   
2     AFG  Afghanistan                                  Kandahar JOL   

      gppd_idnr  capacity_mw  latitude  longitude primary_fuel other_fuel1  \
0  GEODB0040538         33.0    32.322     65.119        Hydro         NaN   
1    WKS0070144         10.0    31.670     65.795        Solar         NaN   
2    WKS0071196         10.0    31.623     65.792        Solar         NaN   

  other_fuel2  ... estimated_generation_gwh_2013  \
0         NaN  ...                        123.77   
1         NaN  ...                         18.43   
2         NaN  ...                         18.64   

   estimated_generation_gwh_2014 estimated_generation_gwh_2015  \
0                         162.90                         97.39   
1                          17.48                         18.25   
2                          17.58                         19.10   

  estimated_generation_gwh_2016 estimated_generation_gwh_2017  \
0                        137.76                        119.50   
1                         17.70                         18.29   
2                         17.62                         18.72   

  estimated_generation_note_2013 estimated_generation_note_2014  \
0                       HYDRO-V1                       HYDRO-V1   
1                SOLAR-V1-NO-AGE                SOLAR-V1-NO-AGE   
2                SOLAR-V1-NO-AGE                SOLAR-V1-NO-AGE   

   estimated_generation_note_2015  estimated_generation_note_2016  \
0                        HYDRO-V1                        HYDRO-V1   
1                 SOLAR-V1-NO-AGE                 SOLAR-V1-NO-AGE   
2                 SOLAR-V1-NO-AGE                 SOLAR-V1-NO-AGE   

   estimated_generation_note_2017  
0                        HYDRO-V1  
1                 SOLAR-V1-NO-AGE  
2                 SOLAR-V1-NO-AGE  

[3 rows x 36 columns]

In [5]:
df = df[['country_long', 'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2', 'other_fuel3', 'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018', 'generation_gwh_2019', 'estimated_generation_gwh_2013', 'estimated_generation_gwh_2014', 'estimated_generation_gwh_2015','estimated_generation_gwh_2016', 'estimated_generation_gwh_2017']]
df.head(3)

country_long  latitude  longitude primary_fuel other_fuel1 other_fuel2  \
0  Afghanistan    32.322     65.119        Hydro         NaN         NaN   
1  Afghanistan    31.670     65.795        Solar         NaN         NaN   
2  Afghanistan    31.623     65.792        Solar         NaN         NaN   

  other_fuel3  generation_gwh_2013  generation_gwh_2014  generation_gwh_2015  \
0         NaN                  NaN                  NaN                  NaN   
1         NaN                  NaN                  NaN                  NaN   
2         NaN                  NaN                  NaN                  NaN   

   generation_gwh_2016  generation_gwh_2017  generation_gwh_2018  \
0                  NaN                  NaN                  NaN   
1                  NaN                  NaN                  NaN   
2                  NaN                  NaN                  NaN   

   generation_gwh_2019  estimated_generation_gwh_2013  \
0                  NaN                         123.77   
1                  NaN                          18.43   
2                  NaN                          18.64   

   estimated_generation_gwh_2014  estimated_generation_gwh_2015  \
0                         162.90                          97.39   
1                          17.48                          18.25   
2                          17.58                          19.10   

   estimated_generation_gwh_2016  estimated_generation_gwh_2017  
0                         137.76                         119.50  
1                          17.70                          18.29  
2                          17.62                          18.72

In [26]:
df = df[['country_long', 'primary_fuel', 'estimated_generation_gwh_2017']]
df = df.fillna(0)
df.head(3)

country_long primary_fuel  estimated_generation_gwh_2017
0  Afghanistan        Hydro                         119.50
1  Afghanistan        Solar                          18.29
2  Afghanistan        Solar                          18.72

In [27]:
df = df.groupby(['country_long', 'primary_fuel']).sum().reset_index()

In [28]:
df = df.rename(columns={'country_long': 'country','estimated_generation_gwh_2017': 'gwh' })

In [29]:
df.head(3)

country primary_fuel     gwh
0  Afghanistan          Gas    0.00
1  Afghanistan        Hydro  791.07
2  Afghanistan        Solar   37.01

In [30]:
df = pd.pivot_table(df, index='country', columns='primary_fuel')
df = df.fillna(0)
df.head(3)

gwh                                                          \
primary_fuel Biomass Coal Cogeneration       Gas Geothermal    Hydro Nuclear   
country                                                                        
Afghanistan      0.0  0.0          0.0      0.00        0.0   791.07     0.0   
Albania          0.0  0.0          0.0      0.00        0.0  4608.91     0.0   
Algeria          0.0  0.0          0.0  63380.37        0.0    30.82     0.0   

                                                                            
primary_fuel  Oil Other Petcoke    Solar Storage Waste Wave and Tidal Wind  
country                                                                     
Afghanistan   0.0   0.0     0.0    37.01     0.0   0.0            0.0  0.0  
Albania       0.0   0.0     0.0     0.00     0.0   0.0            0.0  0.0  
Algeria       0.0   0.0     0.0  1157.62     0.0   0.0            0.0  0.0

In [31]:
df.to_csv('./df_world.csv')

In [8]:
df['country_long'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antarctica',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Central African Republic', 'Chile', 'China', 'Colombia', 'Congo',
       'Costa Rica', 'Cote DIvoire', 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Democratic Republic of the Congo', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'Fiji', 'Finland', 'France', 'French Guiana', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Iran', 'Ira

In [9]:
asia_countries = ['Azerbaijan', 'Bangladesh', 'Bhutan', 'China', 'India', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan','Laos', 'Lebanon', 'Liberia', 'Libya', 'Malaysia', 'Mongolia', 'Myanmar', 'Nepal', 'North Korea', 'Pakistan', 'Palestine', 'Philippines', 'Qatar', 'Singapore', 'South Korea', 'Sri Lanka', 'Syrian Arab Republic', 'Taiwan', 'Tajikistan','Thailand', 'Turkmenistan', 'United Arab Emirates','Uzbekistan', 'Vietnam', 'Yemen']

In [62]:
df_asia = df[df['country_long'].isin(asia_countries)][['country_long', 'primary_fuel', 'generation_gwh_2013','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016','generation_gwh_2017','generation_gwh_2018', 'estimated_generation_gwh_2013','estimated_generation_gwh_2014','estimated_generation_gwh_2015', 'estimated_generation_gwh_2016', 'estimated_generation_gwh_2017']]
df_asia = df_asia.fillna(0)
df_asia.head(3)

country_long primary_fuel  generation_gwh_2013  generation_gwh_2014  \
925   Azerbaijan          Gas                  0.0                  0.0   
926   Azerbaijan          Oil                  0.0                  0.0   
927   Azerbaijan          Gas                  0.0                  0.0   

     generation_gwh_2015  generation_gwh_2016  generation_gwh_2017  \
925                  0.0                  0.0                  0.0   
926                  0.0                  0.0                  0.0   
927                  0.0                  0.0                  0.0   

     generation_gwh_2018  estimated_generation_gwh_2013  \
925                  0.0                            0.0   
926                  0.0                            0.0   
927                  0.0                            0.0   

     estimated_generation_gwh_2014  estimated_generation_gwh_2015  \
925                            0.0                            0.0   
926                            0.0                            0.0   
927                            0.0                            0.0   

     estimated_generation_gwh_2016  estimated_generation_gwh_2017  
925                            0.0                         665.51  
926                            0.0                        1260.90  
927                            0.0                         803.21

In [63]:
len(df_asia['country_long'].unique())

38

In [64]:
df_asia = df_asia.groupby(['country_long', 'primary_fuel']).sum().reset_index()

In [65]:
df_asia = df_asia[['country_long', 'primary_fuel', 'estimated_generation_gwh_2013','estimated_generation_gwh_2014','estimated_generation_gwh_2015', 'estimated_generation_gwh_2016', 'estimated_generation_gwh_2017']]

In [66]:
df_asia = df_asia.rename(columns={'country_long': 'country','estimated_generation_gwh_2013': '2013','estimated_generation_gwh_2014': '2014','estimated_generation_gwh_2015': '2015', 'estimated_generation_gwh_2016': '2016','estimated_generation_gwh_2017': '2017' })

In [67]:
len(df_asia['country'].unique())

38

In [61]:
df_asia.to_csv('./df_asia.csv')